<a href="https://colab.research.google.com/github/AmanNiyad/Financial_Tweet_ATAE-LSTM/blob/main/Financial_Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
# from keras_self_attention import SeqSelfAttention
import math
import nltk
from nltk.corpus import stopwords
import tensorflow as tf

In [6]:
from google.colab import files
uploaded = files.upload()

Saving sent_train.csv to sent_train.csv
Saving sent_valid.csv to sent_valid.csv


In [3]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import io
df = pd.read_csv("sent_train.csv")
stop_words = set(stopwords.words('english'))
df.head()

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral https://t....,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0


In [5]:
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
url_regex = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
handle_regex= "^@?(\w){1,15}$"


df['text']=df['text'].str.replace(url_regex, '')
df['text']=df['text'].str.replace(handle_regex, '')

df.head()

<ipython-input-5-16ff4947b1b4>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text']=df['text'].str.replace(url_regex, '')
<ipython-input-5-16ff4947b1b4>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text']=df['text'].str.replace(handle_regex, '')


,text,label
0,$BYND - JPMorgan reels expectations Beyond Meat,0
1,$CCL $RCL - Nomura points bookings weakness Ca...,0
2,"$CX - Cemex cut Credit Suisse, J.P. Morgan wea...",0
3,$ESS: BTIG Research cuts Neutral,0
4,$FNKO - Funko slides Piper Jaffray PT cut,0


In [6]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df['text'] = df.text.apply(lemmatize_text)

df.head()

,text,label
0,$BYND - JPMorgan reel expectation Beyond Meat,0
1,$CCL $RCL - Nomura point booking weakness Carn...,0
2,"$CX - Cemex cut Credit Suisse, J.P. Morgan wea...",0
3,$ESS: BTIG Research cut Neutral,0
4,$FNKO - Funko slide Piper Jaffray PT cut,0


In [7]:
pos = 0
neg = 0
for i in range(df.shape[0]):
    if df.iloc[i]['label'] == 1:
        pos = pos + 1
    elif df.iloc[i]['label'] == 0:
        neg = neg + 1
neu = df.shape[0]-(pos+neg)
print("Percentage of text with positive sentiment is "+str(pos/df.shape[0]*100)+"%")
print("Percentage of text with negative sentiment is "+str(neg/df.shape[0]*100)+"%")
print("Percentage of text with neutral sentiment is "+str(neu/df.shape[0]*100)+"%")



Percentage of text with positive sentiment is 20.150895944671486%
Percentage of text with negative sentiment is 15.110552237241958%
Percentage of text with neutral sentiment is 64.73855181808655%


In [8]:
class_weights = {0:4,
                1:3,
                2:1}

In [9]:
train_sentences = df['text'].values
labels = df['label'].values
encoder = LabelEncoder()

train_labels = encoder.fit_transform(labels)

#train_sentences, train_labels = train_test_split(reviews, encoded_labels, test_size=0.2, stratify = encoded_labels)

In [10]:
vocab_size = 5068
oov_tok = ""
max_len = 300
embedding_dim = 100
padding_type = 'post'
trunc_type = 'post'
units = 64
hidden_dim = 64

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_len)

In [11]:
def dot_product_attention(h, k, v):
    scores = tf.matmul(h, k, transpose_b=True) / np.sqrt(h.shape[-1])  # Scaled dot-product
    attention_weights = tf.nn.softmax(scores, axis=-1)  # Apply softmax for weights
    return attention_weights

In [12]:
# model = keras.Sequential([
#     keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
#     keras.layers.Bidirectional(keras.layers.LSTM(64)),
#     keras.layers.Dense(1, activation='tanh'),
#     keras.layers.Flatten(),
#     keras.layers.Activation(activation='softmax'),
#     keras.layers.RepeatVector(units*2),
#     keras.layers.Permute([2,1]),
#     keras.layers.Dropout(0.2),
#     # keras.layers.Dense(3, activation='sigmoid'),
# ])

In [13]:
inputs = keras.Input(shape=300,)
embedded = keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len)(inputs)
lstm_output =  keras.layers.Bidirectional(keras.layers.LSTM(64))(embedded)

query = tf.keras.layers.Dense(units=hidden_dim)(lstm_output)
key = tf.keras.layers.Dense(units=hidden_dim)(lstm_output)

attention_weights = dot_product_attention(query, key, lstm_output)
attention_weights = tf.expand_dims(attention_weights, axis=-1)
context_vector = attention_weights * lstm_output
context_vector = tf.reduce_sum(context_vector, axis=1)

model = keras.Model(inputs=inputs, outputs=context_vector)

In [14]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 300)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 300, 100)             506800    ['input_1[0][0]']             
                                                                                                  
 bidirectional (Bidirection  (None, 128)                  84480     ['embedding[0][0]']           
 al)                                                                                              
                                                                                                  
 dense (Dense)               (None, 64)                   8256      ['bidirectional[0][0]']   

In [15]:
num_epochs = 5
history = model.fit(train_padded, train_labels, epochs=num_epochs,verbose=1, validation_split=0.2, class_weight=class_weights)

Epoch 1/5
239/239 [==============================] - 106s 422ms/step - loss: 2.3502 - accuracy: 0.6092 - val_loss: 1.1249 - val_accuracy: 0.1687
Epoch 2/5
239/239 [==============================] - 93s 389ms/step - loss: 1.9249 - accuracy: 0.5980 - val_loss: 1.0902 - val_accuracy: 0.4694
Epoch 3/5
239/239 [==============================] - 93s 390ms/step - loss: 1.9236 - accuracy: 0.6251 - val_loss: 1.0747 - val_accuracy: 0.4694
Epoch 4/5
239/239 [==============================] - 93s 390ms/step - loss: 1.9227 - accuracy: 0.6711 - val_loss: 1.1272 - val_accuracy: 0.4694
Epoch 5/5
239/239 [==============================] - 93s 388ms/step - loss: 1.9248 - accuracy: 0.6730 - val_loss: 1.0809 - val_accuracy: 0.4694


In [20]:
#PREDICTION

In [66]:
df1 = pd.read_csv("sent_valid.csv")

df1['text'] = df1['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
url_regex = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
handle_regex= "^@?(\w){1,15}$"


df1['text']=df1['text'].str.replace(url_regex, '')
df1['text']=df1['text'].str.replace(handle_regex, '')

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df1['text'] = df1.text.apply(lemmatize_text)

<ipython-input-66-a80f6844bc6f>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['text']=df1['text'].str.replace(url_regex, '')
<ipython-input-66-a80f6844bc6f>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df1['text']=df1['text'].str.replace(handle_regex, '')


In [67]:
test_sentences = df['text'].values
labels = df['label'].values
encoder = LabelEncoder()

test_labels = encoder.fit_transform(labels)

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(test_sentences)
word_index = tokenizer.word_index

test_sequences = tokenizer.texts_to_sequences(train_sentences)
test_padded = pad_sequences(train_sequences, padding='post', maxlen=max_len)

In [70]:
prediction = model.predict(test_padded)

299/299 [==============================] - 27s 90ms/step


In [81]:
pred_labels = []
pred = np.asarray(prediction)
pred = np.rint(pred).astype(int)
print(pred)


[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]


In [103]:
sentence = "The stock value is very very good!"
sequence = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequence, padding='post', maxlen=max_len)
prediction = model.predict(padded)

2/2 [==============================] - 0s 38ms/step


In [102]:
pred_labels = []
pred = np.asarray(prediction)
#pred = np.rint(pred).astype(int)
avg = pred.mean()
print(avg)

-0.00052400096
